In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [16]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from redditqa.dataset import load_reddit_dataset
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
import os
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import (
    pipeline,
    TextGenerationPipeline,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSequenceClassification,
    GPTNeoXForCausalLM,
    LlamaForSequenceClassification,
)
from redditqa.dataset import load_reddit_dataset
from transformers.utils.hub import convert_file_size_to_int, get_checkpoint_shard_files
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    set_peft_model_state_dict,
)
from peft.utils import _get_submodules
import peft
import torch
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats import inter_rater as irr
import base64

#### Loading the eval dataset

In [3]:
eval_dataset = load_reddit_dataset("eval")
eval_dataset = eval_dataset.select(range(200))

Loading cached processed dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-cba55e4212677d14.arrow
Loading cached shuffled indices for dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-71c7c533e04253a7.arrow


In [4]:
questions = eval_dataset['question_title']

#### Run models

In [5]:
model_checkpoints = {
    'baseline': 'meta-llama/Llama-2-7b-chat-hf',
    'baseline-nochat': 'meta-llama/Llama-2-7b-hf',
    'checkpoint-2500': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-2500_merged',
    'checkpoint-8500': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-8500_merged',
}

Load the models and make sure they actually are different

In [6]:
# models = {}
# for name, checkpoint_path in model_checkpoints.items():
#     print(f"Loading {name} from {checkpoint_path}")
#     model = AutoModelForCausalLM.from_pretrained(checkpoint_path, torch_dtype=torch.bfloat16)
#     model.config.pad_token_id = model.config.eos_token_id
#     models[name] = model
#
# display(models['baseline'].model.layers[0].self_attn.q_proj.weight)
# display(models['checkpoint-2500'].model.layers[0].self_attn.q_proj.weight)
# display(models['checkpoint-8500'].model.layers[0].self_attn.q_proj.weight)

Generate answers for questions with the models

In [7]:
template = "<|ELIF|> Question: %question\nAnswer: "

def generate_answers(model_path):

    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.config.pad_token_id = model.config.eos_token_id
    model = model.cuda().eval()

    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "min_length": 32,
        "max_length": 128,
    }

    pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device='cuda:0')

    answers = {}

    for question in tqdm(questions):
        prompt = template.replace('%question', question)
        result = pipeline(prompt, **generation_kwargs, return_full_text=False)
        result = result[0]['generated_text']

        answers[question] = result

    return answers


model_answers = {}
for model_name, checkpoint in model_checkpoints.items():
    model_answers[model_name] = generate_answers(checkpoint)

  5%|▌         | 10/200 [00:19<06:04,  1.92s/it]/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1082: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 200/200 [05:17<00:00,  1.59s/it]


#### Reward scores

Now, let's run our reward model on these generations

In [8]:
# Load the reward model
reward_model_checkpoint = "/scratch1/jhoff/checkpoints/reward_llama-2-7b-chat-hf/checkpoint-3000_merged"

tokenizer = AutoTokenizer.from_pretrained(reward_model_checkpoint)

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_checkpoint, 
    num_labels=1, 
    torch_dtype=torch.bfloat16
)

reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device=0,
)

Loading checkpoint shards: 100%|██████████| 3/3 [03:38<00:00, 72.83s/it]


In [9]:
# Apply the reward model to the answers 
template = "<|ELIF|> Question: %question\nAnswer: %answer"

model_rewards = {}
for model_name, model_qas in model_answers.items(): 
    print(f"Applying reward model to answers from {model_name}")
    model_rewards[model_name] = {}
    for question, answer in tqdm(model_qas.items()):
        # Create prompt based on the question and answer
        prompt = template.replace("%question", question).replace("%answer", answer)

        # Apply the reward model to the prompt
        reward = reward_pipe(prompt)[0]["score"]

        # Store the reward
        model_rewards[model_name][question] = reward

model_rewards

Applying reward model to answers from baseline


100%|██████████| 200/200 [00:10<00:00, 18.40it/s]


Applying reward model to answers from baseline-nochat


100%|██████████| 200/200 [00:11<00:00, 18.17it/s]


Applying reward model to answers from checkpoint-2500


100%|██████████| 200/200 [00:09<00:00, 21.77it/s]


Applying reward model to answers from checkpoint-8500


100%|██████████| 200/200 [00:08<00:00, 22.44it/s]


{'baseline': {'How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?': 0.9674102663993835,
  "why aren't CCTV cameras made to produce higher quality footage?": 0.9820137619972229,
  'how is Bill Gates still so rich if he is giving away most of his money?': 0.9609517455101013,
  'How did America become so powerful despite being such a young country (also having powerful enemies in their beginnings)?': 0.9637799859046936,
  'How do they make white gold? Can it be melted down and reused like yellow gold?': 0.976669192314148,
  'Why do humans need to brush their teeth? What did we do before toothpaste?': 0.9664104580879211,
  'How can Americans find the time and money to afford the lifestyle I witnessed in SoCal?': 0.977022647857666,
  'In the US, how did medical/dental/etc. insurance evolve to be tied to your job, instead of being part of a competitive marke

Create a dataframe containing the Answers

In [13]:
dfs = {}

for name, qas in model_answers.items():

    rewards = model_rewards[name]

    df = pd.DataFrame.from_records([
        {'question': q, f'{name}.answer': a, f'{name}.reward': rewards[q]} 
        for q, a in qas.items()
    ])

    dfs[name] = df


df = dfs['baseline']
for name, df_other in dfs.items():
    
    if name == 'baseline':
        continue

    df = df.merge(df_other, on='question')

df

,question,baseline.answer,baseline.reward,baseline-nochat.answer,baseline-nochat.reward,checkpoint-2500.answer,checkpoint-2500.reward,checkpoint-8500.answer,checkpoint-8500.reward
0,How does new city formation works in US? If me...,|<|ELIF|> Question: How does new city formati...,0.967410,1) The state government has no control over th...,0.957278,1. There is a very strong rule of thumb that i...,0.923039,1. Is there a population of at least 4000 peop...,0.953966
1,why aren't CCTV cameras made to produce higher...,🤖 There are several reasons why CCTV cameras a...,0.982014,1. Because it's expensive\n2. Because it's ine...,0.927363,1) They are not designed to be used as evidenc...,0.930458,1) Higher quality doesn't necessarily mean bet...,0.939913
2,how is Bill Gates still so rich if he is givin...,Bill Gates is still rich because he has a sig...,0.960952,"1) He is still the richest man alive, even aft...",0.789234,1) He still owns quite a bit of Microsoft stoc...,0.924142,"He has a lot of money, and he is giving away ...",0.896251
3,How did America become so powerful despite bei...,|<|ELIF|>| America's rise to power can be att...,0.963780,"1) It was a strong, young country with powerfu...",0.867934,"1. Lots of land, resources, and climate for fa...",0.957912,1. Strong economy\n\n2. Strong military\n\n3. ...,0.916110
4,How do they make white gold? Can it be melted ...,White gold is an alloy of gold that is mixed ...,0.976669,"White gold is an alloy of gold with silver, z...",0.968856,24k is 99.9% pure. 14k is 91.6% pure. Making 1...,0.960361,"1. Gold is mixed with a metal, typically silve...",0.964321
...,...,...,...,...,...,...,...,...,...
195,"In a roll of duct tape, how does the glue stic...",The glue on a roll of duct tape is applied to...,0.960361,1. The glue is on the outside. 2. The tape is ...,0.800692,[LINK]-3238/\n\n,0.890294,1. They use a special paper that is coated wit...,0.880797
196,How do popular social media apps become popula...,0<|ELIF|> Popular social media apps often star...,0.981167,0-1-9-15-1\n|ELIF|> Question: What is the diff...,0.896251,"1. They're free. \n\n2. They're not Facebook, ...",0.876634,1. A large company or investment bank (such as...,0.935346
197,Are we fishing at too high of a rate? Are we g...,We are currently fishing at a sustainable rat...,0.966410,"100% Unlikely, the recent stocking of 30,000 f...",0.866133,1) We are not fishing at a rate that is sustai...,0.874935,1. We've already fished at too high of a rate....,0.953966
198,What is a freemason and why is it such a secre...,A Freemason is a member of a fraternal organi...,0.982557,1.A freemason is a member of a secret society....,0.897695,1. It is not a secret society\n\n2. It is a so...,0.938124,Freemasonry is a fraternity. It's a group of ...,0.874077


Show mean reward scores for different checkpoints

In [15]:
df[['baseline.reward', 'baseline-nochat.reward', 'checkpoint-2500.reward', 'checkpoint-8500.reward']].mean()

baseline.reward           0.968004
baseline-nochat.reward    0.912144
checkpoint-2500.reward    0.918719
checkpoint-8500.reward    0.912536
dtype: float64

Review an example

In [41]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(pd.DataFrame(df.iloc[0]))

,0
question,"How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?"
baseline.answer,"|<|ELIF|> Question: How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?\nIn the United States, new city formation typically involves a series of steps and requirements that must be met"
baseline.reward,0.96741
baseline-nochat.answer,"1) The state government has no control over this. You are basically free to do what you want on your land.\n2) If you have 50 people living on your land, you can probably get some kind of designation from the state, but it would have no legal significance.\n3) If you want to live in a self-cont"
baseline-nochat.reward,0.957278
checkpoint-2500.answer,"1. There is a very strong rule of thumb that if you can drive to the center of the city in less than 30 minutes, it is not a city.\n\n2. The state will not give you a city charter until you have a population of over 5000 people.\n\n3. People move there.\n\n"
checkpoint-2500.reward,0.923039
checkpoint-8500.answer,"1. Is there a population of at least 4000 people living within 4mi of a central point.\n2. Is a permanent settlement of at least 20 people living within 4mi of a central point.\n\nIf you meet both of those criteria, then it is a town.\n\nIf you meet both of those"
checkpoint-8500.reward,0.953966


#### Human annotators 

Generate sheet to compare models for human annotators. We focus on the baseline and the two checkpoints after 2500 and 8500 steps. We compare to the left and make sure that the order is randomized

In [27]:
def encode_metadata(metadata):
    metadata = repr(metadata)
    metadata = metadata.encode('utf-8')
    metadata = base64.b64encode(metadata)
    return metadata

In [62]:
def create_comparison(col1, col2): 

    # Create a new dataframe with the two columns
    comparisons = []

    for i, row in df.iterrows():

        question = row['question']
        answer1 = row[col1]
        answer2 = row[col2]

        do_swap = np.random.choice([True, False])


        comparison_row = {
            'question': question,
            'answer1': answer1 if not do_swap else answer2,
            'answer2': answer2 if not do_swap else answer1,
            'metadata': {
                'rand1': np.random.randint(0, 1e5),
                'do_swap': do_swap,
                'answer1': col1 if not do_swap else col2,
                'answer2': col2 if not do_swap else col1, 
                'rand2': np.random.randint(0, 1e5)
            },
        }
        comparison_row['metadata_encoded'] = encode_metadata(comparison_row['metadata'])

        comparisons.append(comparison_row)

    df_comparisons = pd.DataFrame.from_records(comparisons)
    return df_comparisons

#with pd.option_context('display.max_colwidth', None):
#    display(create_comparison('baseline.answer', 'checkpoint-2500.answer'))

In [60]:
create_comparison('baseline.answer', 'checkpoint-2500.answer')\
    .to_excel('Model Comparison - Baseline vs Checkpoint-2500 (n200).xlsx')

create_comparison('checkpoint-2500.answer', 'checkpoint-8500.answer')\
    .to_excel('Model Comparison - Checkpoint-2500 vs Checkpoint-8500 (n200).xlsx')